# Load models

In [21]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("hf_token")
login(token=secret_value)

In [23]:
model_names = [
    "batoulnn/arabert_author_style_class_checkpoint",
    "batoulnn/camelbert_authorsclass6",
    "MuhammadHelmy/GATE-AraBert-AuthId",
    "MuhammadHelmy/xlm-roberta-base-arabic-AuthId",
]

loaded_models_and_tokenizers = []

for model_name in model_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
    model.eval()
    loaded_models_and_tokenizers.append((model, tokenizer))

print(f"Loaded {len(loaded_models_and_tokenizers)} models and tokenizers.")

Loaded 4 models and tokenizers.


In [ ]:
# model_weights = [0.26, 0.25, 0.247, 0.243] # The sum is 1.0

# Load data

In [24]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [36]:
# Replace 'path/to/your/dataset.csv' with the actual path to your dataset file in Google Drive
dataset_path = '/kaggle/input/arageneval-data/valid_data_cleaned_topics.csv'
test_path = '/kaggle/input/arageneval-data/test_data_cleaned.csv'
df = pd.read_csv(dataset_path, index_col="Unnamed: 0")
test_df = pd.read_csv(test_path, index_col="Unnamed: 0")


display(df.head())
print(f"Dataset size: {len(df)}")
print(f"Number of unique authors: {df['author'].nunique()}") # Assuming 'author' is the column with author labels

,id,text_in_author_style,author,avg_word_length,text_size,detected_language,cleaned_text,normalized_text,topic,topic_keywords
Unnamed: 0,,,,,,,,,,
0,3835,من طفل في الخمسين\n\nعمري ما احتفلت أو حفلت بع...,يوسف إدريس,4.485623,1747,ar,من طفل في الخمسين\n\nعمري ما احتفلت أو حفلت بع...,طفل الخمسين عمري احتفلت او حفلت بعيد ميلادي كن...,-1,"['الادب', 'الاسلامية', 'الاسلام', 'الدين', 'ال..."
1,3836,ذلك الزمن العام هو العداد العام الذي\n\nدام يع...,يوسف إدريس,4.578171,1922,ar,ذلك الزمن العام هو العداد العام الذي\n\nدام يع...,الزمن العام العداد العام دام يعد السنين والايا...,-1,"['الادب', 'الاسلامية', 'الاسلام', 'الدين', 'ال..."
2,3837,مصر الغنية المثقفة المصنِّعة، والعرب\n\nوقد\n\...,يوسف إدريس,4.816770,1903,ar,مصر الغنية المثقفة المصنعة، والعرب\n\nوقد\n\nأ...,مصر الغنية المثقفة المصنعة والعرب وقد احالوا ب...,-1,"['الادب', 'الاسلامية', 'الاسلام', 'الدين', 'ال..."
3,3838,ولأنها غريبة وراودتني فيها عن الناس وعن الحياة...,يوسف إدريس,4.733333,1747,ar,ولأنها غريبة وراودتني فيها عن الناس وعن الحياة...,ولانها غريبة وراودتني الناس وعن الحياة وعن نفس...,-1,"['الادب', 'الاسلامية', 'الاسلام', 'الدين', 'ال..."
4,3839,وليس ما ذكرته مرارة ولا ندمًا؛ فقد كان لا يمكن...,يوسف إدريس,4.537367,1580,ar,وليس ما ذكرته مرارة ولا ندما؛ فقد كان لا يمكن ...,وليس ذكرته مرارة ندما فقد يمكن ان يحدث الا حدث...,-1,"['الادب', 'الاسلامية', 'الاسلام', 'الدين', 'ال..."


Dataset size: 4157
Number of unique authors: 21


In [26]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['author'])

# For inverse transform later
author_names = label_encoder.classes_

# Ensemble Implementation

In [27]:
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, classification_report

In [28]:
def predict_proba(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=-1)
    return probs.squeeze().cpu().numpy()

In [29]:
def ensemble_predict(text):
    final_probs = None
    num_models = len(loaded_models_and_tokenizers)
    for model, tokenizer in loaded_models_and_tokenizers:
        probs = predict_proba(model, tokenizer, text)
        if final_probs is None:
            final_probs = probs
        else:
            final_probs += probs
    final_probs /= num_models # Calculate the simple average
    predicted_class = final_probs.argmax()
    return predicted_class, final_probs

# Predictions

In [30]:
predictions = []
for text in df['cleaned_text']:
    pred_class, probs = ensemble_predict(text)
    predictions.append(pred_class)

print('Finished Prediction!')

Finished Prediction!


In [31]:
# Transform the true labels to numerical labels using the fitted LabelEncoder
true_labels_encoded = label_encoder.transform(df['author'])

accuracy = accuracy_score(true_labels_encoded, predictions)
print("Accuracy:", accuracy)
print(classification_report(true_labels_encoded, predictions, target_names=author_names))

Accuracy: 0.923983642049555
                   precision    recall  f1-score   support

        أحمد أمين       0.87      0.91      0.89       246
  أحمد تيمور باشا       0.80      0.93      0.86        57
        أحمد شوقي       0.74      0.79      0.77        58
    أمين الريحاني       0.82      0.85      0.84       142
       ثروت أباظة       0.67      0.27      0.38        90
 جبران خليل جبران       0.83      1.00      0.91        30
      جُرجي زيدان       0.98      0.95      0.97       327
         حسن حنفي       0.99      0.99      0.99       548
        روبرت بار       0.89      0.98      0.93        82
       سلامة موسى       0.94      1.00      0.97       119
          طه حسين       0.92      0.98      0.95       255
عباس محمود العقاد       0.96      0.91      0.93       267
 عبد الغفار مكاوي       0.94      0.96      0.95       396
     غوستاف لوبون       0.92      0.98      0.95       150
       فؤاد زكريا       0.80      1.00      0.89       125
      كامل كيلاني       0.9

In [38]:
test_preds = []
for text in test_df['cleaned_text']:
    pred_class, probs = ensemble_predict(text)
    test_preds.append(pred_class)

print('Finished Prediction!')

Finished Prediction!


In [39]:
final_labels = label_encoder.inverse_transform(test_preds)
print(final_labels)

preds = pd.DataFrame({
    'id': range(len(final_labels)),
    'label': final_labels
})
display(preds)

preds.to_csv("predictions.csv", index=False)

['أحمد تيمور باشا' 'حسن حنفي' 'روبرت بار' ... 'نجيب محفوظ' 'حسن حنفي'
 'جُرجي زيدان']


,id,label
0,0,أحمد تيمور باشا
1,1,حسن حنفي
2,2,روبرت بار
3,3,أمين الريحاني
4,4,روبرت بار
...,...,...
8408,8408,أحمد أمين
8409,8409,غوستاف لوبون
8410,8410,نجيب محفوظ
8411,8411,حسن حنفي
